In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from datetime import datetime
import time
from __future__ import print_function
import json
from pymongo import MongoClient

In [2]:
# wesley
ckey = 'NdQdQm2OALKTJgtCr9mz8SEAK'
csecret= '2GVLXTHcIy3htHqD5NEIX1IGVuTNjiHMHNEXT2OTTlvjP50hKw'
atoken= '980527124133109766-R2NKGIqJjrUvkOALRw7bdzAHRa4FV18'
asecret= 'r5Px3c5gLtXVuNjJI53qoImV1PW6CeXwRyv8gnjqwTzt0'

In [22]:
# keran
ckey = 'ok5bMdzCP7UMJMZUMm9tLZdtg'
csecret= 'J9V0m78943Q4dDzQGzOiSpVWex2H1rjSRR5TCAQ2g5zUt7m0Gm'
atoken= '980478205151842304-TQ8AWkaDSh5XtLvsa2hyWfAJONcdHxp'
asecret= 'SbH7zQJnFdQRF6qzUv1BWaUgAjksGdrPdnOGB2Ft8vByl'

In [3]:
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [9]:
user=["@BeijingAir"]
original=["air #pollution","空气污染","#airpollution","वायु प्रदुषण","la pollution de l'air","la #pollution","air #pollution","l'air #pollution","वायु प्रदुषण"]
category_1=["forest fire","forest fires"]
category_2=["water poisoning","water contamination"]
category_3=["flood flash","flood water","flood warning"]
category_4=["earthquake"]
category_5=["oil spill","pipeline spill","tarsands spill","tankers spill", "fossilfuel spill","petroleum spill"]
category_6=["acid rain","toxic rain","mildew home","mildew infested","mildew basement","mildew removal","mildew growth","mildew inspection","mold flood","mildew flood"]
category_7=["adelgid","aphid","beetle","earwig","insect","locust","louse","moth","psyllid","termites","termite bites","bugs","bug","bug bites","bugs bites"]

In [18]:
category_8="""allergy
allergins
pollen
dander
dust
allergy + cough
allergins + cough
pollen + cough
dander + cough
dust + cough
allergy + sneeze
allergins + sneeze
pollen + sneeze
dander + sneeze
dust + sneeze
allergy + asthma
allergins + asthma
pollen + asthma
dander + asthma
dust + asthma
allergy + respiratory
allergins + respiratory
pollen + respiratory
dander + respiratory
dust + respiratory
allergy + lung
allergins + lung
pollen + lung
dander + lung
dust + lung""".split("\n")
category_8=[word.replace("+","") for word in category_8]
category_8=[word.replace("  "," ") for word in category_8]

In [19]:
# category_17=["@beijingair"]
category_9=["dust"]
category_10=["bed bugs"]

In [20]:
keywordlist= user +original+category_1+category_2+category_3+category_4+category_5+category_6+category_7+category_8+category_9+category_10
keywordlist

['@BeijingAir',
 'air #pollution',
 '空气污染',
 '#airpollution',
 'वायु प्रदुषण',
 "la pollution de l'air",
 'la #pollution',
 'air #pollution',
 "l'air #pollution",
 'वायु प्रदुषण',
 'forest fire',
 'forest fires',
 'water poisoning',
 'water contamination',
 'flood flash',
 'flood water',
 'flood warning',
 'earthquake',
 'oil spill',
 'pipeline spill',
 'tarsands spill',
 'tankers spill',
 'fossilfuel spill',
 'petroleum spill',
 'acid rain',
 'toxic rain',
 'mildew home',
 'mildew infested',
 'mildew basement',
 'mildew removal',
 'mildew growth',
 'mildew inspection',
 'mold flood',
 'mildew flood',
 'adelgid',
 'aphid',
 'beetle',
 'earwig',
 'insect',
 'locust',
 'louse',
 'moth',
 'psyllid',
 'termites',
 'termite bites',
 'bugs',
 'bug',
 'bug bites',
 'bugs bites',
 'allergy',
 'allergins',
 'pollen',
 'dander',
 'dust',
 'allergy cough',
 'allergins cough',
 'pollen cough',
 'dander cough',
 'dust cough',
 'allergy sneeze',
 'allergins sneeze',
 'pollen sneeze',
 'dander sneeze

In [ ]:
# MONGO_HOST= 'mongodb://localhost/twitterdb'  # assuming you have mongoDB installed locally
                                             # and a database called 'twitterdb'
WORDS = keywordlist
client = MongoClient()
# Use twitterdb database. If it doesn't exist, it will be created.
db = client.twitterdb
class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False

    def on_data(self, data):
        #This is the meat of the script...it connects to your mongoDB and stores the tweet
        try:
#                 client = MongoClient(MONGO_HOST)


            # Decode the JSON from Twitter
            datajson = json.loads(data)

            #use tweet id as mongodb id
            datajson["_id"]=datajson.pop("id")

            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
#             print("Tweet collected at " + str(created_at))

            #insert the data into the mongoDB into a collection called twitter_search
            #if twitter_search doesn't exist, it will be created.
            db.twitter_search.insert_one(datajson)
        except Exception as e:
            print(e)

auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

Tracking: ['@BeijingAir', 'air #pollution', '空气污染', '#airpollution', 'वायु प्रदुषण', "la pollution de l'air", 'la #pollution', 'air #pollution', "l'air #pollution", 'वायु प्रदुषण', 'forest fire', 'forest fires', 'water poisoning', 'water contamination', 'flood flash', 'flood water', 'flood warning', 'earthquake', 'oil spill', 'pipeline spill', 'tarsands spill', 'tankers spill', 'fossilfuel spill', 'petroleum spill', 'acid rain', 'toxic rain', 'mildew home', 'mildew infested', 'mildew basement', 'mildew removal', 'mildew growth', 'mildew inspection', 'mold flood', 'mildew flood', 'adelgid', 'aphid', 'beetle', 'earwig', 'insect', 'locust', 'louse', 'moth', 'psyllid', 'termites', 'termite bites', 'bugs', 'bug', 'bug bites', 'bugs bites', 'allergy', 'allergins', 'pollen', 'dander', 'dust', 'allergy cough', 'allergins cough', 'pollen cough', 'dander cough', 'dust cough', 'allergy sneeze', 'allergins sneeze', 'pollen sneeze', 'dander sneeze', 'dust sneeze', 'allergy asthma', 'allergins asthm

E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003583900373868545 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003584652706308096 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003585159411613698 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003585668453359616 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003585668939800576 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003585667534745600 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003585920245719042 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003586170725457920 }
E11000 duplicate key error collection: twitterdb.twitter_search index: _id_ dup key: { : 1003586164492660738 }
E